# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [2]:
# Import the necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#Option 1. Just the basic function

def scrape_books(min_rating, max_price):
    base_url = 'http://books.toscrape.com/catalogue/'
    books = []
    
    for page in range(1, 51):  # Assuming there are 50 pages
        url = f'http://books.toscrape.com/catalogue/page-{page}.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        book_list = soup.select('.product_pod')
        
        for book in book_list:
            title = book.h3.a['title']
            price = float(book.select_one('.price_color').text[2:])
            rating = book.p['class'][1]
            rating_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
            star_rating = rating_dict[rating]
            
            if star_rating >= min_rating and price <= max_price:
                book_url = base_url + book.h3.a['href']
                book_response = requests.get(book_url)
                book_soup = BeautifulSoup(book_response.text, 'html.parser')
                genre = book_soup.select('ul.breadcrumb li a')[2].text
                upc = book_soup.select_one('th:contains("UPC") + td').text
                availability = book_soup.select_one('th:contains("Availability") + td').text.strip()
                description_tag = book_soup.select_one('#product_description ~ p')
                description = description_tag.text if description_tag else 'No description available'
                
                books.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': star_rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description
                })
    
    df = pd.DataFrame(books)
    return df

# Execute the function to scrape data for books with a minimum rating of 4.0 and above and a maximum price of £20
df_books = scrape_books(4, 20)
print(df_books)


In [17]:
#Option 2, handling exceptions and errors

def scrape_books(min_rating, max_price):
    base_url = 'http://books.toscrape.com/catalogue/'
    books = []
    
    for page in range(1, 51):  # Assuming there are 50 pages
        try:
            url = f'http://books.toscrape.com/catalogue/page-{page}.html'
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            book_list = soup.select('.product_pod')
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            continue
        
        for book in book_list:
            try:
                title = book.h3.a['title']
                price = float(book.select_one('.price_color').text[2:])
                rating = book.p['class'][1]
                rating_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
                star_rating = rating_dict[rating]

                if star_rating >= min_rating and price <= max_price:
                    book_url = base_url + book.h3.a['href']
                    book_response = requests.get(book_url)
                    book_response.raise_for_status()
                    book_soup = BeautifulSoup(book_response.text, 'html.parser')
                    genre = book_soup.select('ul.breadcrumb li a')[2].text
                    upc = book_soup.select_one('th:contains("UPC") + td').text
                    availability = book_soup.select_one('th:contains("Availability") + td').text.strip()
                    description_tag = book_soup.select_one('#product_description ~ p')
                    description = description_tag.text if description_tag else 'No description available'

                    books.append({
                        'UPC': upc,
                        'Title': title,
                        'Price (£)': price,
                        'Rating': star_rating,
                        'Genre': genre,
                        'Availability': availability,
                        'Description': description
                    })
            except Exception as e:
                print(f"Error processing book: {e}")
                continue
    
    df = pd.DataFrame(books)
    return df

# Execute the function to scrape data for books with a minimum rating of 4.0 and above and a maximum price of £20
df_books = scrape_books(4, 20)
print(df_books)


                 UPC                                              Title  \
0   ce6396b0f23f6ecc                                        Set Me Free   
1   6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2   6be3beb0793a53e7                                     Sophie's World   
3   657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4   51653ef291ab7ddc                                    This One Summer   
..               ...                                                ...   
70  9c96cd1329fbd82d                                    The Zombie Room   
71  b78deb463531d078                                    The Silent Wife   
72  4280ac3eab57aa5d                                  The Girl You Lost   
73  29fc016c459aeb14              The Edge of Reason (Bridget Jones #2)   
74  19fec36a1dfb4c16  A Spy's Devotion (The Regency Spies of London #1)   

    Price (£)  Rating               Genre             Availability  \
0       17.46       5        